In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
from IPython.display import display

from openmm import *
from simtk.unit import *
import MDAnalysis as md
import nglview as ng
from sys import stdout

In [4]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import MDAnalysis as md
import nglview as ng
from sys import stdout

In [8]:
# These files are already preloaded to your ``data`` folder

pdb0_file = "data/villin_water.pdb"
pdb1_file = "data/polyALA.pdb"
pdb2_file = "data/polyGLY.pdb"
pdb3_file = "./polyGV.pdb"

In [10]:
# 1. loading initial coordinates
pdb = PDBFile(pdb3_file)

In [11]:
# 2.  choosing a forcefield parameters
ff = ForceField("amber10.xml")
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

In [12]:
# 3. Choose parameters of the experiment: temperature, pressure, box size, solvation, boundary conditions, etc...
temperature = 300*kelvin
frictionCoeff = 1/picosecond
time_step = 0.002*picoseconds
total_steps = 400*picoseconds / time_step

In [13]:
# 4. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

In [14]:
# 5. Run simulation, saving coordinates time to time
## 5a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [15]:
## 5b. Minimize energy
simulation.minimizeEnergy()

In [17]:
## 5c. Save coordinates to dcd file and energies to a standard output console
simulation.reporters.append(DCDReporter("./polyGV_traj.dcd", 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,
                                              temperature=True, progress=True, totalSteps=total_steps))

In [18]:
## 5d. Run
simulation.step(total_steps)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)"
2.5%,5000,3829.4159507751465,294.08636427982566
5.0%,10000,3802.703638523817,297.03404969540327
7.5%,15000,3512.748431161046,314.96779693449565
10.0%,20000,3701.0789977796376,296.5762193748673
12.5%,25000,3584.767472870648,309.4853840579445
15.0%,30000,3490.185280010104,315.9398873243975
17.5%,35000,3521.8836820051074,307.8527137311294
20.0%,40000,3562.424770478159,319.0299744571546
22.5%,45000,3590.708060803823,286.91851800585135
25.0%,50000,3599.541151098907,303.9113385289278
27.5%,55000,3525.222045958042,293.56321394764626
30.0%,60000,3529.8517248928547,298.9462395148083
32.5%,65000,3464.23014562577,314.7979061552405
35.0%,70000,3511.2270562574267,317.0457306443005
37.5%,75000,3524.2890021651983,305.7365386453871
40.0%,80000,3479.6729863397777,324.39871069674746
42.5%,85000,3474.8231246620417,314.73366408737434
45.0%,90000,3438.1747557148337,298.2237036964964
47.5%,95000,3526.193709731102,293.71715830089624
50.0%,

In [20]:
## Visualization
# Let's look at the trajectory
#6. Visualization
sys = md.Universe(pdb3_file, "./polyGV_traj.dcd")
ng.show_mdanalysis(sys, gui=True)

/home/media/data2/user_home/dbsejrgus226/miniconda3/envs/MD/lib/python3.8/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


NGLWidget(max_frame=199)

In [21]:
# End-to-end distance
# analysis of end-to-end distance
#choose terminal atoms

N_terminus = sys.select_atoms("resid 1 and name N")
C_terminus = sys.select_atoms("resid 25 and name C")

# go through the whole trajectory and compute distance between them for every frame
dist = []
for frame in sys.trajectory:
    dist.append(np.linalg.norm(N_terminus.positions - C_terminus.positions))

# the result is in the dist array
dist = np.array(dist)

In [23]:
# Number of hydrogen bonds
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis ## module for analysis of hydrogen bonds

## compute information about hbonds and write it to the 'hb.timeseries'
hb = HydrogenBondAnalysis(sys)
hb.run()

## print information for the first 10 frames
for frame in hb.timeseries[:10]:
    print(frame)

NoDataError: This Universe does not contain charge information